# M&M model VEM updates

In [1]:
dat = readRDS('/home/gaow/Documents/GTExV8/Thyroid.Lung.FMO2.filled.rds')
attach(dat)

In [2]:
%get X Y --from R

Loading required package: feather


In [3]:
from libgaow.regression_data import RegressionData, MASH
from libgaow.model_mash import PriorMASH
import numpy as np

In [11]:
class MNMASH(RegressionData):
    def __init__(self, X=None, Y=None, Z=None, B=None, S=None, V=None):
        RegressionData.__init__(self, X, Y, Z, B, S)
        self.V = V
        self.alpha0 = None
        self.mu0 = None
        self.Xr0 = np.zeros((self.Y.shape[0], self.Y.shape[1]))
        self.trace_XXt = np.sum(np.square(self.X), axis = 1)
        self.elbo = []
    
    def set_prior(self, U, grid, pi):
        self.U = U
        self.grid = grid
        self.pi = pi
        prior = PriorMASH(self)
        prior.expand_cov()

    def fit(self, niter=100, L=5, calc_elbo=False):
        if self.V is None:
            self.V = np.cov(self.Y, rowvar=False)
        self.alpha0 = np.zeros((L, self.X.shape[1]))
        self.mu0 = np.zeros((L, self.X.shape[1], self.Y.shape[1]))
        for i in range(niter):
            self._calc_update()
            if calc_elbo:
                self._calc_elbo()

    def _calc_update(self):
        for l in range(self.alpha0.shape[0]):
            self.Xr0 -= self.X @ (self.alpha0[l,:] * self.mu0[l,:,:])
            self.alpha0[l,:], self.mu0[l,:,:] = self._calc_single_snp(self.Y - self.Xr0)
            self.Xr0 += self.X @ (self.alpha0[l,:] * self.mu0[l,:,:])

    def _calc_single_snp(self, R):
        mash = MASH(R, self.X)
        mash.fit()
        bf = np.exp(mash.l10bf)
        return bf/np.sum(bf), mash.post_mean_mat        

    def _calc_elbo(self):
        pass

In [12]:
class ELBOMNM:
    def __init__(self, data):
        self.data = data

In [18]:
np.cov(Y, rowvar = False)

array([[ 0.27297989,  0.01692897],
       [ 0.01692897,  0.11091973]])

In [15]:
Y.shape

NameError at line 1 of 
>>> Y.shape
name 'Y' is not defined
